# Create json files for shared trips choropleth (Figs 3a, 3b)
Fig 3a - number of requested (and completed) shared trips  
Fig 3b - proportion of requested shared trips that were matched up  

Omit the geom !  and rename columns to `area_s_cd` and `prop`  

Queries stored in `~/Documents/PROJECTS/VFH/COT_PAGE/QUERIES/` as:  
`Fig3a_prop_shared_trips_by_nn.sql`  
`Fig3b_prop_actual_shared_by_nn.sql`  

**refs**    

* https://geoffboeing.com/2015/10/exporting-python-data-geojson/  
Huom: `with open(output_filename, 'wb') as output_file:` should be `'w'` not `'wb'`  
* geoJSON minifier: https://jsonformatter.org/json-minify  

## Fig 3a Query (Fig3a_prop_shared_trips_by_nn.sql)
From https://github.com/CityofToronto/bdit_vfh/blob/master/notebooks/vfh_final.ipynb, **Proportion of Shared Ride Trips by Neighbourhood, September 2018, Exhibit 3-25**:

```
CREATE TABLE cnangini.fig3a_shared_trips AS
WITH daily_ave AS (

    SELECT sum(count) as count, pickup_neighbourhood 
    FROM ptc.trip_data_agg_neighbourhood_pool
    WHERE pickup_datetime > '2018-08-31'
    GROUP BY  pickup_neighbourhood
),

daily_ave_all AS (
    SELECT sum(count) as count, pickup_neighbourhood 
    FROM ptc.trip_data_agg_neighbourhood
    WHERE pickup_datetime > '2018-08-31'
    GROUP BY  pickup_neighbourhood
)

SELECT pickup_neighbourhood, ROUND(100 * a.count/b.count) AS prop --, geom
FROM daily_ave a
LEFT JOIN daily_ave_all b USING (pickup_neighbourhood)
LEFT JOIN gis.neighbourhood ON pickup_neighbourhood = area_s_cd::integer
WHERE pickup_neighbourhood IS NOT NULL
ORDER by pickup_neighbourhood;

-- Add dummy rows for nn 141, 142
INSERT INTO cnangini.fig3a_shared_trips (pickup_neighbourhood, prop) 
VALUES (141, NULL);

INSERT INTO cnangini.fig3a_shared_trips (pickup_neighbourhood, prop) 
VALUES (142, NULL);

ALTER TABLE cnangini.fig3a_shared_trips
RENAME COLUMN pickup_neighbourhood TO area_s_cd;

```

_**Note:** table `cnangini.fig3a_shared_trips` to be deleted from schema; saved as `~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/Fig3a_shared_trips.csv`_

## Fig 3b Query (Fig3b_prop_actual_shared_by_nn.sql)
**made by Raph à-la-volée :**  

```
CREATE TABLE cnangini.fig3b_matched_shared_trips AS
 WITH raw_data AS (
     SELECT trip_data_new_geom.pickup_datetime::date AS pickup_datetime,
            trip_data_new_geom.pickup_neighbourhood,
            date_part('hour'::text, trip_data_new_geom.pickup_datetime) AS hr,
            trip_data_new_geom.ptctripid,
	 		count(*) OVER (PARTITION BY ptctripid) AS num_shared
       FROM ptc.trip_data_new_geom
      WHERE (trip_data_new_geom.provider_id = ANY (ARRAY[28608558, 1608409])) 
             AND trip_data_new_geom.dropoff_datetime IS NOT NULL 
             AND trip_data_new_geom.pickup_datetime IS NOT NULL
             AND trip_data_new_geom.pickup_datetime >= '2018-09-01 00:00:00'::timestamp without time zone 
             AND trip_data_new_geom.dropoff_datetime >= '2018-09-01 00:00:00'::timestamp without time zone
)

SELECT pickup_neighbourhood, 
   100 * SUM(CASE WHEN num_shared > 1 THEN 1 ELSE 0 END) --Count of picked up passengers in this neighbourhood who   actually shared their trip
   / COUNT(*) -- number of completed shared trip requests for this neighbourhood
   AS prop -- proportion_matched_shared_trips, geom
FROM raw_data
LEFT JOIN gis.neighbourhood ON pickup_neighbourhood = area_s_cd::integer
WHERE pickup_neighbourhood IS NOT NULL
GROUP BY pickup_neighbourhood --, geom
ORDER BY pickup_neighbourhood;

-- Add dummy rows for nn 141, 142
INSERT INTO cnangini.fig3b_matched_shared_trips (pickup_neighbourhood, proportion_matched_shared_trips) 
VALUES (141, NULL);

INSERT INTO cnangini.fig3b_matched_shared_trips (pickup_neighbourhood, proportion_matched_shared_trips) 
VALUES (142, NULL);

ALTER TABLE cnangini.fig3b_matched_shared_trips
RENAME COLUMN pickup_neighbourhood TO area_s_cd;

```

_**Note:** table `cnangini.fig3b_matched_shared_trips` to be deleted from schema; saved as `~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/Fig3b_matched_shared_trips.csv`_

# Save tables to csv file in pgAdmin

Go to pgAdmin and export `cnangini.fig3a_shared_trips` and `cnangini.fig3b_matched_shared_trips` to csv.  

=> save in `~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/` as  
`Fig3a_shared_trips_by_nn.csv` and  
`fig3b_matched_shared_trips_by_nn.csv`  

# Convert csv to json with online tool

https://www.csvjson.com/csv2json  

# Minify json with online tool

https://jsonformatter.org/json-minify  

```
[{"area_s_cd":1,"prop":35},{"area_s_cd":2,"prop":39},{"area_s_cd":3,"prop":40},{"area_s_cd":4,"prop":36},{"area_s_cd":5,"prop":38},{"area_s_cd":6,"prop":36},{"area_s_cd":7,"prop":33},{"area_s_cd":8,"prop":39},{"area_s_cd":9,"prop":40},{"area_s_cd":10,"prop":32},{"area_s_cd":11,"prop":32},{"area_s_cd":12,"prop":31},{"area_s_cd":13,"prop":34},{"area_s_cd":14,"prop":38},{"area_s_cd":15,"prop":32},{"area_s_cd":16,"prop":38},{"area_s_cd":17,"prop":49},{"area_s_cd":18,"prop":43},{"area_s_cd":19,"prop":37},{"area_s_cd":20,"prop":37},{"area_s_cd":21,"prop":37},{"area_s_cd":22,"prop":40},{"area_s_cd":23,"prop":41},{"area_s_cd":24,"prop":43},{"area_s_cd":25,"prop":45},{"area_s_cd":26,"prop":44},{"area_s_cd":27,"prop":46},{"area_s_cd":28,"prop":45},{"area_s_cd":29,"prop":46},{"area_s_cd":30,"prop":49},{"area_s_cd":31,"prop":53},{"area_s_cd":32,"prop":48},{"area_s_cd":33,"prop":45},{"area_s_cd":34,"prop":43},{"area_s_cd":35,"prop":48},{"area_s_cd":36,"prop":47},{"area_s_cd":37,"prop":48},{"area_s_cd":38,"prop":47},{"area_s_cd":39,"prop":47},{"area_s_cd":40,"prop":43},{"area_s_cd":41,"prop":46},{"area_s_cd":42,"prop":44},{"area_s_cd":43,"prop":47},{"area_s_cd":44,"prop":47},{"area_s_cd":45,"prop":45},{"area_s_cd":46,"prop":39},{"area_s_cd":47,"prop":43},{"area_s_cd":48,"prop":42},{"area_s_cd":49,"prop":40},{"area_s_cd":50,"prop":46},{"area_s_cd":51,"prop":47},{"area_s_cd":52,"prop":43},{"area_s_cd":53,"prop":42},{"area_s_cd":54,"prop":47},{"area_s_cd":55,"prop":47},{"area_s_cd":56,"prop":51},{"area_s_cd":57,"prop":56},{"area_s_cd":58,"prop":56},{"area_s_cd":59,"prop":52},{"area_s_cd":60,"prop":48},{"area_s_cd":61,"prop":44},{"area_s_cd":62,"prop":46},{"area_s_cd":63,"prop":55},{"area_s_cd":64,"prop":54},{"area_s_cd":65,"prop":57},{"area_s_cd":66,"prop":48},{"area_s_cd":67,"prop":51},{"area_s_cd":68,"prop":56},{"area_s_cd":69,"prop":51},{"area_s_cd":70,"prop":61},{"area_s_cd":71,"prop":65},{"area_s_cd":72,"prop":65},{"area_s_cd":73,"prop":66},{"area_s_cd":74,"prop":68},{"area_s_cd":75,"prop":71},{"area_s_cd":76,"prop":71},{"area_s_cd":77,"prop":69},{"area_s_cd":78,"prop":71},{"area_s_cd":79,"prop":68},{"area_s_cd":80,"prop":67},{"area_s_cd":81,"prop":70},{"area_s_cd":82,"prop":67},{"area_s_cd":83,"prop":60},{"area_s_cd":84,"prop":67},{"area_s_cd":85,"prop":62},{"area_s_cd":86,"prop":58},{"area_s_cd":87,"prop":52},{"area_s_cd":88,"prop":52},{"area_s_cd":89,"prop":46},{"area_s_cd":90,"prop":51},{"area_s_cd":91,"prop":58},{"area_s_cd":92,"prop":62},{"area_s_cd":93,"prop":63},{"area_s_cd":94,"prop":63},{"area_s_cd":95,"prop":66},{"area_s_cd":96,"prop":64},{"area_s_cd":97,"prop":62},{"area_s_cd":98,"prop":58},{"area_s_cd":99,"prop":57},{"area_s_cd":100,"prop":57},{"area_s_cd":101,"prop":59},{"area_s_cd":102,"prop":55},{"area_s_cd":103,"prop":55},{"area_s_cd":104,"prop":61},{"area_s_cd":105,"prop":48},{"area_s_cd":106,"prop":62},{"area_s_cd":107,"prop":62},{"area_s_cd":108,"prop":57},{"area_s_cd":109,"prop":59},{"area_s_cd":110,"prop":55},{"area_s_cd":111,"prop":50},{"area_s_cd":112,"prop":51},{"area_s_cd":113,"prop":41},{"area_s_cd":114,"prop":43},{"area_s_cd":115,"prop":49},{"area_s_cd":116,"prop":35},{"area_s_cd":117,"prop":37},{"area_s_cd":118,"prop":37},{"area_s_cd":119,"prop":40},{"area_s_cd":120,"prop":41},{"area_s_cd":121,"prop":44},{"area_s_cd":122,"prop":40},{"area_s_cd":123,"prop":38},{"area_s_cd":124,"prop":40},{"area_s_cd":125,"prop":39},{"area_s_cd":126,"prop":38},{"area_s_cd":127,"prop":37},{"area_s_cd":128,"prop":34},{"area_s_cd":129,"prop":33},{"area_s_cd":130,"prop":29},{"area_s_cd":131,"prop":27},{"area_s_cd":132,"prop":32},{"area_s_cd":133,"prop":24},{"area_s_cd":134,"prop":34},{"area_s_cd":135,"prop":33},{"area_s_cd":136,"prop":39},{"area_s_cd":137,"prop":35},{"area_s_cd":138,"prop":40},{"area_s_cd":139,"prop":37},{"area_s_cd":140,"prop":30},{"area_s_cd":141,"prop":""},{"area_s_cd":142,"prop":""}]```

# Format manually into an object with `requested` and `matched` keys:  
Copy the minified arrays from above step into the object keys:  

```
{"requested": [],
"matched":[]
}
```